In [1]:
!pip uninstall timm transformers accelerate peft -y

Found existing installation: timm 1.0.15
Uninstalling timm-1.0.15:
  Successfully uninstalled timm-1.0.15
Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
Found existing installation: accelerate 1.8.1
Uninstalling accelerate-1.8.1:
  Successfully uninstalled accelerate-1.8.1
Found existing installation: peft 0.15.2
Uninstalling peft-0.15.2:
  Successfully uninstalled peft-0.15.2


In [2]:
try:
    import timm
    import transformers
    print(timm.__version__ , transformers.__version__)
except Exception as e:
    print(e)

No module named 'timm'


In [3]:
import sys
sys.path.append("/kaggle/input/pip-vlmfs")
# sys.path.append("/kaggle/input/pip-quantization")

import timm
import torch
import torch.nn as nn
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer 
from typing import List, Union , Dict

In [4]:
lora_path   = "/kaggle/input/lmweights/lmweights_lora"
model_path  = "/kaggle/input/gemma3"
save_path   = "/kaggle/working"

language_model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    device_map="cuda:0",
    torch_dtype = torch.bfloat16,
    low_cpu_mem_usage=True,
)

2025-08-03 01:16:47.844030: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754183808.082475      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754183808.150797      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
sys.path.append("/kaggle/input/pip-vlmfs/dir2")
from peft import LoraConfig , get_peft_model , PeftModel , PeftConfig

In [6]:
try:
    import timm
    import transformers
    print(timm.__version__ , transformers.__version__)
except Exception as e:
    print(e)

1.0.17 4.53.2


In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
# model_final = PeftModel.from_pretrained(language_model, lora_path, is_trainable=True , autocast_adapter_dtype=False)
model_final = PeftModel.from_pretrained(language_model, lora_path, is_trainable=True)
model_final.print_trainable_parameters()

trainable params: 338,165,760 || all params: 5,777,604,032 || trainable%: 5.8530


In [8]:
l = []
for c in model_final.parameters():
    l.append(c.dtype)

set(l)

{torch.bfloat16, torch.float32}

In [9]:
from safetensors import safe_open

p = "/kaggle/input/lmweights/lmweights_lora/adapter_model.safetensors"

precisions = []
with safe_open(p, framework="pt", device="cpu") as f:
    for key in f.keys():
        tensor = f.get_tensor(key)
        precisions.append(tensor.dtype)

set(precisions)

{torch.bfloat16}

In [10]:
merged_model = model_final.merge_and_unload()
merged_model = merged_model.to(dtype=torch.bfloat16)

merged_model.save_pretrained(save_path , torch_dtype=torch.bfloat16)
tokenizer.save_pretrained(save_path)

('/kaggle/working/tokenizer_config.json',
 '/kaggle/working/special_tokens_map.json',
 '/kaggle/working/tokenizer.model',
 '/kaggle/working/added_tokens.json',
 '/kaggle/working/tokenizer.json')

In [11]:
import shutil
import os

files = [
    "/kaggle/input/gemma3/preprocessor_config.json",
    "/kaggle/input/gemma3/processor_config.json"   
]

destination = "/kaggle/working"  

os.makedirs(destination, exist_ok=True)

for file in files:
    shutil.copy(file, destination)
    print(f"Copied {file} to {destination}")


Copied /kaggle/input/gemma3/preprocessor_config.json to /kaggle/working
Copied /kaggle/input/gemma3/processor_config.json to /kaggle/working


In [12]:
!touch siglip.py